# Feature Selection
Artur Back de Luca

GEIA - Grupo de estudos em Inteligência Aritificial

## Content:
<br>

1. The context;
2. The problem;
3. Possible solutions;
4. Feature selection approaches: Filter, Wrapper and Embedded methods;

Occam's razor (law of parsimony)

_"Entities should not be multiplied without necessity."_

When presented with competing hypotheses that make the same predictions, one should select the solution with the fewest assumptions

# 1. Context

<h3>What is a feature</h3>

<i>In machine learning and pattern recognition, a **feature** (or an attribute) is an individual measurable property or characteristic of a phenomenon being observed. <a href="#References">[1]</a></i>

<p style="font-size:0.6em; text-align:right">1. The context</p>
<p style="font-size:0.8em;">
    1997: very few domains operated with more than 40 features <a style="font-size:0.6em;" href="#References">[1]</a></p>
    <ul style="margin:0">
        <li style="font-size:0.6em;">Soybean - 1988 (35 features)</li>
        <li style="font-size:0.6em;">Letter Recognition - 1991 (16 features)</li>
        <li style="font-size:0.6em;">Lung Cancer - 1992 (32 features)</li>
    </ul>

<p style="font-size:0.8em;">2009: ImageNet - 14 million images with 256x256 pixels (+196k features) each, and more than 20,000 categories <a style="font-size:0.6em;" href="#References">[2]</a></p>
<p style="font-size:0.8em;">2010: The Wikipedia Corpus - almost 1.9 billion words from more than 4 million articles <a style="font-size:0.6em;" href="#References">[3]</a></p>
<p style="font-size:0.8em;">2011: Cancer detection based on gene expression (e.g.: Colon dataset - 2,000 features) <a style="font-size:0.6em;" href="#References">[4]</a></p>
<br>
<center><img width="800" src="./figures/number_of_attributes_growth.png"/></center>
<p style="font-size:0.6em; text-align:right">Source: <a href="#References">[5]</a></p>

ImageNet: 256x256x3 = 196608

Wikipedia: 1.9 billion words, not distinct, of course - the Second Edition of the 20-volume Oxford English Dictionary, published in 1989, contains full entries for 171,476 words in current use, and 47,156 obsolete words.

# 2. The problem

<p style="font-size:0.6em; text-align:right">2. The problem</p>
<h3>Too many features, too many problems</h3>
<p style="font-size:0.8em"><i>What problems arise with too many features?</i></p>

---

1. require longer training times*

2. jeopardize human interpretability*

3. worsen prediction quality due to sample size effects

4. potentially increase overfitting

<p style="font-size:0.6em; text-align:right">* Considered self-explainatory, will not be explained below</p>

<p style="font-size:0.6em; text-align:right">2. The problem</p>
<h4>3. Worsen prediction quality due to sample size effects</h4>
<p style="font-size:0.8em"><i>Couldn't a predictor simply disregard irrelevant features?</i></p>

---

To answer this, we will have to resort to some statistical learning theory, exploring the ways of estimating functional dependency from a given collection of data.

<p style="font-size:0.6em; text-align:right">2.3. The problem: Worsen accuracy due to sample size effects</p>
<h5>Statistical Learning Theory</h5>

Let $X \in \mathbb R^p$ be a input random vector and $Y\in \mathbb R$ be an output random variable, with joint distribution $P(X,Y)$.

The task of learning aims at finding a function $f(X)$ for predicting $Y$ given values of input $X$. This structure requires a *loss function* $L(Y, f(X))$ for identifying and penalizing errors in prediction. With this structure, we can define a criterion for choosing a suitable $f$ known as the *Statistical Risk ($R$)*.

$$
\begin{equation}
\begin{split}
\text{R}(f) & =  \mathbb{E} \thinspace L(Y, f(X)) \\
 & = \int L(y, f(x))\thinspace P(dx,dy) \\
 & = \mathbb{E}_{X}\mathbb{E}_{Y|X} [L(Y, f(X))|X]
\end{split}
\end{equation}
$$
<p style="text-align:right;">Source: <a href="#References">[5]</a></p>

This criterion tells us how well, on average, the predictor $f$ performs with respect to the chosen loss function

Instead of working with the joint distribution, we can condition the Statistical Risk on $X$
$$
\begin{equation}
\begin{split}
\text{R}(f) & = \int\int [L(y, f(x))|x]\thinspace P(dy)\thinspace P(dx) \\
 & = \mathbb{E}_{X}\mathbb{E}_{Y|X} [L(Y, f(X))|X]
\end{split}
\end{equation}
$$

Finally, what we seek is a $f(X)$ which minimizes the the Risk:
<br><br>
$$f(X)_{opt}= argmin_c \mathbb{E}_{X}\mathbb{E}_{Y|X}[L(Y,c)|X]$$

<p style="font-size:0.6em; text-align:right">2.3. The problem: Worsen accuracy due to sample size effects</p>
The optimal solution will be different depending on the metric used

<h5>Regression</h5>
<h6>Mean Squared Error (MSE)</h6>
<br>
$$
\begin{equation}
\begin{split}
f(X)_{opt} & = argmin_c \mathbb{E}_{X}\mathbb{E}_{Y|X}[(Y-c)^2|X] \\
 & = \mathbb{E}(Y|X=x)
\end{split}
\end{equation}
$$
known as the conditional mean or expectation - the "average" value over an arbitrarily large number of occurrences
<h6>Mean Absolute Error (MAP)</h6>
<br>
$$
\begin{equation}
\begin{split}
f(X)_{opt} & = argmin_c \mathbb{E}_{X}\mathbb{E}_{Y|X}[|Y-c|\thinspace|X] \\
 & = median(Y|X=x)
\end{split}
\end{equation}
$$
or the conditional median of the distribution


<h3>Regression loss functions for SLT</h3>

**MSE**: the most preferable option, due to its ease of computation of minimum, since it is differentiable. However it is more sensitive to outliers as a big difference becomes even larger by squaring them. 

**MAP**: its optimal estimates are more robust than those for the conditional mean. However, MAP has discontinuities in their derivatives, which have hindered their widespread use.

<p style="font-size:0.6em; text-align:right">2.3. The problem: Worsen accuracy due to sample size effects</p>
<h5>Classification</h5>
<h6>0-1 loss</h6>
$$
\begin{equation}
\begin{split}
f(X)_{opt} & = argmin_c \mathbb{E}_{X}\mathbb{E}_{Y|X}[I(f(X),Y)] \\
 & = \underset{y \in Y}{\max}P(y|X)
\end{split}
\end{equation}
$$
where ${\displaystyle I}$ is the indicator function:

$$I :=\begin{cases}
0\text{, if } f(X) = Y\\
1\text{, otherwise}\\
\end{cases}$$

The decider is also known as the *Optimal Bayes classifier*.

If the joint probability $P(X,Y)$ is known and the classification decision is optimal.
<center><img src="./figures/bayes_error.svg" width="900" align="center"/></center>
This doesn't mean that there are no errors, rather than the lowest error achievable, resulted by noise among distributions.


<p style="font-size:0.6em; text-align:right">2.3. The problem: Worsen accuracy due to sample size effects</p>
If the probability distribution for the problem was known, the classifier wouldn't be affected by adding more of features.

If such features carried the slightest contributions, it is shown that the Bayes optimal classifier tends to zero as the number of features approach infinity.

<center><img src='./figures/trunk.png' width="800"/></center>

However, the probability distribution used is an estimation, based on the finite set of samples, causing a **peaking phenomenon**: the prediction accuracy increases with the number of features, but soon reaches a peak, in which the noise becomes larger than the separability increase caused by the new feature.
<p style="font-size:0.6em; text-align:right;">Source: <a href="#References">[6]</a></p>

This is often called the peaking phenomenon. Some ideas evolved around it are discussed in the great pattern recognition blog by Ela Pekalska and Bob Duin called [37steps](http://37steps.com/about/):
 - [Peaking summarized](http://37steps.com/2545/peaking-summarized/)
 - [Peaking paradox](http://37steps.com/2279/the-peaking-paradox/)
 - [Trunk’s example of the peaking phenomenond](http://37steps.com/2448/trunks-example/)
 - [The curse of dimensionality](http://37steps.com/2349/curse-of-dimensionality/)
 

<p style="font-size:0.6em; text-align:right">2. The problem</p>
<h4> 4. potentially increase overfitting</h4>

<p>As the number of features increase, the observations become more sparse within the feature space.</p>

<br>
<table>
    <tr>
        <td>
            <img src='./figures/exp_curse_of_overfitting.svg' width="1500"/>
        </td>    
        <td style="font-size:3em; width: 700px; word-wrap: break-word;">
            <p>Having the observations further apart makes it difficult for the estimator to generalize, increasing its variance, i.e. relying on the specific observations to produce new predictions, causing overfitting.</p>
        </td>
    </tr>
</table>

Consequences for classical:

**Non-parametric (Local methods)**: methods such as the $k$ *nearest neighboors*, as the examples become increasingly sparse, the approximation of the conditional expectation by taking the average of its nearest neighboors becomes correspondingly worse as the query point distance itself from the known examples. Additionally, in high dimensions, as the datapoints become more spread apart, their distances become more uniform, making it dificult for the algorithm to decide which data points are more relevant to region of interest [Source](https://www.youtube.com/watch?v=dZrGXYty3qc&list=PLXP3p2MXX_ahmhsMiP5YWtc7IcABeFOGH&index=2)

**Parametric methods**:  In very high dimensional spaces, there is more than one plane that can be fitted to your data, and without proper type of regularization can cause the model to behave very poorly. . Collinearity often results in overfitting, i.e. in a too efficient modelling of learning samples without model generalization ability. [[7]]()

More examples on  check the answer on [stackexchange](https://stats.stackexchange.com/questions/186184/does-dimensionality-curse-effect-some-models-more-than-others)

What causes these problems?

<p style="font-size:0.8em; text-align:right"><i>The curse of dimensionality</i></p>

<p style="font-size:0.6em; text-align:right">2. The problem</p>
<h3>The curse of dimensionality</h3>
<br>
<table>
    <tr>
        <td>
            <img src='./figures/exp_curse_of_dimensionality.svg' width="700"/>
        </td>    
        <td style="font-size:3em; width: 700px; word-wrap: break-word;">
            <p>Increasing the number of factors to be taken into consideration requires an exponential growth of observations <a href="#References">[5]</a><br><br>
           However, having a small number of samples means that many regions of the feature space are never observed</p>
        </td>
    </tr>
</table>

# 3. How to overcome this problem?

<p style="font-size:0.8em; text-align:right"><i>Ans.: Dimensionality reduction</i></p>

<h2>Dimensionality reduction</h2>

The process of reducing the number attributes under consideration by obtaining a set of principal variables.

It can be carried out through two distinct techniques: feature *extraction* and *selection*

<h2>Feature extraction</h2>

Feature extraction projects the original high-dimensional features to a new feature space with low dimensionality through a combination of the original features.

 - **Advantages**: It can generate meaning with incomprehensible features
 - **Disavantages**: It may lose physical meanings of comprehensible features
 
 <br>
 
**Types:**
 1. Ad hoc techniques;
 2. Data transformation techniques;
 3. Deep learning techniques.

<p style="font-size:0.6em; text-align:right">3.1 Feature extraction</p>

<h3>1. Ad hoc techniques</h3>

Consists on relying on user expertise to derive meaningful relationship between features
 - **Advantages**: Interpretability
 - **Disavantages**: Time-consuption and prior knowledge

A fundamental step in this process is to perform an exploratory analysis - [*Facets*](https://ai.googleblog.com/2017/07/facets-open-source-visualization-tool.html) may be a helpful tool.
<center><img src="https://2.bp.blogspot.com/-Kab341D9VYI/WWz0NrlzH_I/AAAAAAAAB5E/BkIxG4WnADgQTmAFxLSw2zoAuPvIRw6igCLcBGAs/s1600/image1.gif"/></center>

<p style="font-size:0.6em; text-align:right">3.1 Feature extraction</p>

<h2>2. Data transformation and clustering techniques</h2>

Instead of relying solely on the expert's knownledge on the data, several techniques have been already developed to for the purpose of reducing the dimensions of a problem

<table>
    <tr>
        <td>
            <a href="https://lvdmaaten.github.io/tsne/">
                <img src="./figures/tsne.png" width="400">
            </a>
        </td>
        <td>
            <a href="https://commons.wikimedia.org/wiki/File:Classical_Multidimensional_Scaling_(MDS)_analysis_performed_on_the_1,719_samples.png">
                <img src="./figures/mds.png" width="400"/>
            </a>
        </td>
        <td>
            <a href="https://austingwalters.com/pca-principal-component-analysis/">
                <img src="./figures/pca.png" width="400">
            </a>
        </td>
    </tr>
    <tr>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://youtu.be/NEaUSP4YerM">
                    t-SNE
                </a>
            </p>
        </td>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://youtu.be/GEn-_dAyYME">
                    MDS
                </a>
            </p>
        </td>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://youtu.be/FgakZw6K1QQ">
                    PCA
                </a>
            </p>
        </td>
    </tr>
    <tr>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html">
                    scikit.learn
                </a>
            </p>
        </td>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html">
                    scikit.learn
                </a>
            </p>
        </td>
        <td>
            <p style="font-size: 2em; text-align:center">
                <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">
                    scikit.learn
                </a>
            </p>
        </td>
    </tr>
</table>

<p style="font-size:0.6em; text-align:right">3.1 Feature extraction</p>
<h3>3. Deep learning techniques</h3>

Deep learning architectures successively combine feature transformations in order to generate higher hierarchical representations that are more associated with the given task in hand.
<p style="font-size:0.8em">Examples: Stacked autoenconders, Deep Belief Nets, Sparse filtering and Convolutional layers</p>

In [2]:
from IPython.core.display import HTML

zoom = 0.9
width = 900
height = 800
url = "https://distill.pub/2018/building-blocks/#AttributionGroups"

html = (
    '''
        <style type="text/css">
        #frame {
            zoom: {zoom};
            -moz-transform:scale({zoom});
            -moz-transform-origin: 0 0;
            -o-transform: scale({zoom});
            -o-transform-origin: 0 0;
            -webkit-transform: scale({zoom});
            -webkit-transform-origin: 0 0;
        }
        </style>
        <iframe
            height={height}
            width={width}
            id="frame"
            src={url}
            scrolling="no"
        ></iframe>
        <a style="font-size:0.6em" href="{url}">Source</a>
    '''
    .replace("{zoom}", str(zoom))
    .replace("{width}", str(width))
    .replace("{height}", str(height))
    .replace("{url}", url)
)


In [3]:
HTML(html)

<h2>Feature selection</h2>

Feature selection addresses the task of finding a subset of $M$ features from a set of $N$ features, $M < N$, such that the value of a criterion function is optimized over all subsets of size $M$ either:
1. improving prediction quality;
2. decreasing the size of the structure without significantly decreasing performance; or
3. maintaining the original distribution. 

<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[11]</a></p>

---

 - **Advantages**: It retains the physical meaning of features
 - **Disavantages**: It cannot generate new meaning using the original features

<p style="font-size:0.6em; text-align:right">3.2 Feature selection</p>
<h2> What to select?</h2>

A good subset of features should include a subset of the relevant features that optimizes some performance function. The relevance of features can be classified as:
 
 - Strongly relevant: those that if removed handicap the prediction quality of the feature set;
 - Weakly relevant: those that if removed handicap the prediction quality for a particular feature subset;
 - Irrelavant features: those that are not neither strongly or weakly relevant.
 
 <p style="text-align:right; font-size:0.6em">Source: <a href="#References">[12]</a></p>

<h1>4. Feature selection methods</h1>
<br><br>

1. Filter methods;
2. Wrapper methods;
3. Embedded methods.

Disclaimer: This is just the tip of the iceberg
 
 - There are many more techniques than those here presented;
 - Feature selection methods shown are designed for the assumption that the features are independent (*Flat features*). For some cases (e.g. Image Recognition), this assumption does not hold. Check [[13]](#Reference) for a survey on *Strucutred Features* as well.

<p style="font-size:0.6em; text-align:right">4. Feature selection methods</p>
<h3> Filter methods</h3>

Filter methods are independent of any algorithms and are applied before  learning. They rely on characteristics of data to assess feature importance thus discarding less relevant features.

<br>
<center><img src="./figures/filter_methods.svg"/></center>
<br> 




 - **Advantages**: Filter methods are typically computationally efficient

 - **Disavantages**: Due to the lack of a specific learning algorithm guiding the feature selection phase, the selected features may not be optimal; Univariate methods do not capture redundancy.

A typical filter method consists of two steps:
1. Feature importance is ranked according to some feature evaluation criteria
2. Lowly ranked features are filtered out either by a threshold of acceptance or by a limiting number of features to be selected.

Some of the evaluation criteria are:
1. Correlation-based Criteria
2. Fisher Score
3. Information Gain

<p style="font-size:0.6em; text-align:right">3.2.1 Filter methods</p>

<h3>1. Correlation Criteria</h3>

One of the simplest criteria used is the Pearson correlation coefficient:

$$R(i) = \frac{cov(X_i, Y)}{\sqrt{var(X_i)var(Y)}}$$

where $X_i$ is the $i_{th}$ feature. 

Although useful, the  coefficient fails to detect important non-linear dependecies. To overcome this is the user may perform non-linear transformations onto the input, or use different correlation criteria, such as the [Spearman's rank correlation](https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient) or [Kendall's Tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient). 

---

Python implementation: [`scikit-learn.feature_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression) (Pearson) and [`microsoft azure`](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/feature-selection-modules) (Pearson, Kendall and Spearman)

Additionally, the relationship described by such coefficients can be misleading, as demonstrated by [Anscombe's quartet](https://en.wikipedia.org/wiki/Anscombe's_quartet)

<center><img width="900" src="https://upload.wikimedia.org/wikipedia/commons/e/ec/Anscombe%27s_quartet_3.svg"/></center>

Here all of the relationships shown possess the same correlation coefficient, however, it is evident that they do not share the same relationship between variables. This is an important concept to exemplify the limitations of a coefficient and how far can we trust it 

<p style="font-size:0.6em; text-align:right">3.2.1 Filter methods</p>

<h3>2. Fisher Score</h3>

Features with high quality should assign similar values to instances in the same class and different values to instances from different classes. With this intuition, the score for the $i_{th}$ feature $S_i$ will be calculated by Fisher Score as, 
$$S_i = \frac{\sum^K_{j=0}n_j(\mu_{ij}-\mu_{i})^2}{\sum^K_{j=0}n_j\thinspace var(X_{ij})^2}$$
where $\mu_{ij}$ and $\mu_{j}$ are the mean of the $i_{th}$ feature and the mean of the $j_{th}$ class of the $i_{th}$ feature, respectively, while $K$ is the total number of features and $n_j$ is the number of instances in the $j_{th}$ class on $X_i$.

---

Python implementation: [`scikit-feature`](https://github.com/jundongl/scikit-feature)
<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[13]</a></p>

<p style="font-size:0.6em; text-align:right">3.2.1 Filter methods</p>

<h3>3. Information Gain (Mutual information)</h3>

Information Gain expresses the dependence of one variable to another, quantifying how much knowing a feature reduces uncertainty about the outcome.

$$I_i = \sum_{x_i}\sum_yP(X=x_i, Y=y)\thinspace log\frac{P(X=x_i,Y=y)}{P(X=x_i)P(Y=y)}$$

where the probabilities used are an estimation derived from the training set.

If X and Y were independent then MI would be zero and greater than zero if they were dependent.

We can select our features from feature space by ranking their mutual information with the target variable.

---

Python implementation: [`scikit-learn.feature_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_classif.html#sklearn.feature_selection.mutual_info_classif)

<p style="font-size:0.6em; text-align:right">3.2.1 Filter methods</p>
<h4>Addendum: other relevant techniques</h4>

When categorical data is used, other statistical tests are used in order to determine whether the a feature is independent or not from a desired variable. [Chi-Square](https://www.quora.com/How-is-chi-test-used-for-feature-selection-in-machine-learning) test, implemented under [`scikit-learn.feature_selection`](https://scikit-learn.org/stable/modules/feature_selection.html) module

Alternatively, the [RELIEF](https://en.wikipedia.org/wiki/Relief_(feature_selection)) is used to determine consistent dependency between features and classes. Python implementation: [`scikit-rebate`](https://github.com/EpistasisLab/scikit-rebate)

<p style="font-size:0.6em; text-align:right">3.2.1.1 Filter Methods </p>

*Important:* Even though filter methods try to select useful features individually, [[2, pg. 1165]](#References) shows that a variable that is completely useless by itself can provide a significant performance improvement when taken with others.

<p style="font-size:0.6em; text-align:right">3.2 Feature selection</p>
<h3> Wrapper methods</h3>

Wrapper methods rely on the predictive performance of a predefined learning algorithm to evaluate the quality of selected features.

<br>
<center><img src="./figures/wrapper_methods.svg"/></center>
<br> 

 - **Advantages**: Simplicity; Can in principle find the optimal set of features

 - **Disavantages**: Are prone to overfitting; Computationally expensive

A typical wrapper method consists of two steps:
1. search for a subset of features
2. evaluate the selected features. 
3. Repeat *1.* and *2.* until some stopping criteria are satisfied.


Finding the optimal set of features has been shown to be unable to solve in polynomial-time*. [[14]](#References),  therefore using a exaustive search approach is intractable. Moreover, oftentimes the search space in too large ($2^n$) Therefore, different search strategies have been employed:
1. Sequential selection algorithms  
2. Branch-and-bound
3. Metaheuristics

<p style="text-align:right; font-size:0.6em"><b>*</b>Under the assumption that $P ≠ NP$. For more information on computational complexity theory check <a href="https://en.wikipedia.org/wiki/P_versus_NP_problem">this article</a></p>


<p style="font-size:0.6em; text-align:right">3.2.2 Wrapper methods</p>

<h3>1. Sequential Search</h3>

1.  **Sequential Feature Selection (SFS)**<br>
 It Starts with an empty set and gradually adds the feature that most improved the subset predictive performance until it reaches a stopping criteria (predetermined number of features, or the *probing method*).<br><br> 
2. **Sequential Backward Selection (SBS)**<br>
Similar to SFS, but it starts with the complete set of features and gradually removes the feature that gives the lowest decrease in prediction performance.<br><br>
3. **Sequential Floating Forward Selection (SFFS)**<br>
It starts with an empty number and iteratively adds a SFS step followed by a SBS step.<br><br>
4. **Sequential Backward Floating Selection (SBFS)**<br>
It starts with the complete set and iteratively adds SBS step followed by a SFS step<br><br>

The forward methods more efficient than their backward counterpart, as they start with smaller subsets.

---

[Further reading](https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/) and Python implementation: [`mlxtend.feature_selection`](https://rasbt.github.io/mlxtend/user_guide/feature_selection/SequentialFeatureSelector/#api)

<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[15]</a></p>

<p style="font-size:0.6em; text-align:right">3.2.2 Wrapper methods</p>

<h3>2. Branch-and-bound</h3>

Developed in the 1960's the branch and bound is one of the most commonly known algorithms for solving NP-hard problems. 

The subset combinations are arranged in a tree-like structure and bounds on the prediction quality are established throughout the alogrithmic search in order to prevent the exploration of non-optimal candidates

Narendra and Fukunaga (1977) [[16]](#Reference) proposed using branch and bound for feature selection problems. However, an optimal solution can only be guaranteed if monotonicity is satisfied (the addition of a feature to a subset can never decline the prediction quality). This assumption rarely holds in the real-world.

---


Python implementation: [`enusearch`](https://github.com/artur-deluca/enusearch) 

<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[15]</a></p>

<p style="font-size:0.6em; text-align:right">3.2.2 Wrapper methods</p>

<h3>3. Metaheuristics</h3>

Metaheuristics are iterative techniques that use strategies for exploring and exploiting the search space in order to efficiently find near-optimal solutions. [[17]](#References)

There is a plethora of metaheuristics, some of which have been successfully employed in feature selection:



1. Genetic algortihms (GA) [Kudo and Sklansky (2000)](http://www.sciencedirect.com/science/article/abs/pii/S0031320399000412)
2. Particle swarm optimization (PSO) ([Unler and Murat (2010)](http://www.sciencedirect.com/science/article/pii/S0377221710001633))
3. Tabu search (TS) ([Pacheco, Casado and Núñez (2009)](https://www.sciencedirect.com/science/article/abs/pii/S037722170800828X), [Zhang and Sun (2002)](https://www.sciencedirect.com/science/article/abs/pii/S0031320301000462))

As experimentally demonstrated in Unler and Kudo, metaheuristics find more satisfying results as Sequential search techniques oftentimes in less time. However, due to their stochastic nature, oftentimes the results are not consistent as the ones reproduce by the aforementioned techniques

---
Python implementations: GA: [`gaft`](https://github.com/PytLab/gaft) and PSO:[`PSOpt`](https://github.com/artur-deluca/psopt/)


<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[15]</a></p>

Note: Performance assessments are usually done using a validation set or by cross-validation 

<p style="font-size:0.6em; text-align:right">3.2 Feature selection</p>
<h3> Embedded methods</h3>

Embedded methods, are quite similar to wrapper methods since they are also used to optimize the objective function or performance of a learning algorithm or model. The difference to wrapper methods is that an intrinsic model building metric is used during learning

Embedded methods perform variable selection in the process of training and are usually specific to given learning machines.

<br>
<center><img src="./figures/embedded_methods.svg"/></center>
<br> 

 - **Advantages**: Make better usage of data (doesn't split it for evaluation). It is less computationally expensive and less prone to overfitting than wrapper methods 

 - **Disavantages**: More computationally expensive than filter methods.

Some of the embedded techniques are:

1. Built-in selectors
2. Prunning methods
3. Regularization methods

<p style="font-size:0.6em; text-align:right">3.2.3 Embedded methods</p>

<h3>1. Built-in selectors</h3>

Decision tree (CART) algorithms such as the ID3 and C4.5 use feature selection methods in its operations.

The algorithm iterates through every unused attribute of the dataset and calculates the information gain of that attribute. The set is then split by the attribute with highest information gain to produce subsets of data. The algorithm continues to recur on each subset, considering only attributes never selected before.

The algorithm may stop if each example within a certain subset belongs to a class, or any other limitation (given by the number of splits allowed or available attributes)

In these circumstances, these algorithms can sort through features and decide which are the most relevant among them as the model is being built.

---
Python implementation: [`scikit-learn.tree`](https://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart)
<p style="text-align:right; font-size:0.6em"><a href="https://en.wikipedia.org/wiki/Decision_tree_learning">Further reading</a></p>


C4.5 made a number of improvements to ID3. Some of these are:

- Handling both continuous and discrete attributes
- Handling training data with missing attribute values
- Handling attributes with differing costs.
- Pruning trees after creation - C4.5 goes back through the tree once it's been created and attempts to remove branches that do not help by replacing them with leaf nodes.

<p style="font-size:0.6em; text-align:right">3.2.3 Embedded methods</p>

<h3>2. Prunning methods</h3>

<h4>Recursive feature elimination (RFE)</h4>

Recursively eliminates one feature at iteration until predetermined number of features is reached.

It differs from traditional wrapper methods because it uses internal information from the learning algorithm to endorse the elimination.

For linear predictors, RFE eliminates the feature that carries the smallest weight as for decision trees and random forest classifiers, the concept feature importance is used, which can be defined by:
   - *Gini Importance or Mean Decrease in Impurity (MDI)*: the sum over the number of splits (accross all tress) that include the feature, proportionaly to the number of samples it splits.
   - *Permutation Importance or Mean Decrease in Accuracy (MDA)* is assessed for each feature by removing the association between that feature and the target. This is achieved by randomly permuting the values of the feature and measuring the resulting increase in error.

---

Python implementation: [`scikit-learn.feature_selection`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html)

Under scikit-learn, only the Gini importance can used for RFE in tree-based classifiers.

<p style="text-align:right; font-size:0.6em">Source: <a href="https://alexisperrier.com/datascience/2015/08/27/feature-importance-random-forests-gini-accuracy.html">article</a> and <a href="#References">[13]</a></p>

<p style="font-size:0.6em; text-align:right">3.2.3 Embedded methods</p>

<h3>3. Regularization methods</h3>

Regularization methods introduce penalization into the loss function to balance the decision between accuracy and model simplicity. Models with too many parameters have higher penalties, and if a weight associated with a feature sinks below a certain threshold, its value goes to zero.

- Lasso ($L_1$ Regularization)
$$penalty\thinspace(w) = \sum^n_{i=1}\vert w_i\vert$$

- Elastic-net regularization
$$penalty\thinspace(w) = \lambda_1\sum^n_{i=1}\vert w_i\vert + \lambda_2(\sum^n_{i=1}w^2_i), \thinspace\lambda\ge0$$

---

- Adpative Lasso regularization
- Bridge regularization:

---

Python implementation: [`sklearn.linear_model`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model)


<p style="text-align:right; font-size:0.6em">Source: <a href="#References">[13]</a></p>

- L2 regularization (Ridge regression) is not suitable for feature selection since its penalization doesn't reduce coefficients to zero

L1:
- L1 requires a higher coefficient for feature selection than for simple regularization
- When p > n, L1 tends to select n variables before it saturates
- Small nonzero parameters cannot be detected consistently
- High correlations between predictors leads to poor selection performance

Elastic-net:

The quadratic penalty term makes the loss function strongly convex, and it therefore has a unique minimum. 

## Wrap up

**1**  *Do you have domain knowledge?* 
If yes, construct a better set of “ad hoc” features.

**2** *Are your features commensurate?* If no, consider normalizing them.

**3** *Do you suspect interdependence of features?* If yes, expand your feature set by constructing
conjunctive features or products of features, as much as your computer resources allow you.

**4** *Do you need to prune the input variables (e.g. for cost, speed or data understanding reasons)?* If no, construct disjunctive features or weighted sums of features (e.g. clustering or matrix factorization)

**5** Do you need to assess features individually (e.g. to understand their influence on the system
or because their number is so large that you need to do a first filtering)? If yes, use a variable
ranking method; else, do it anyway to get baseline results.

## Wrap up

**6** Do you need a predictor? If no, stop.

**7** Do you suspect your data is “dirty” (has a few meaningless input patterns and/or noisy
outputs or wrong class labels)? If yes, detect the outlier examples using the top ranking
variables obtained in step 5 as representation; check and/or discard them.

**8** Do you know what to try first? If no:
- Take a linear predictor and use a forward selection method with the “probe” method as a stopping criterion or use the 0-norm embedded method. 
- For comparison, following the ranking of step 5, construct a sequence of predictors of same nature using increasing subsets of features. Can you match
or improve performance with a smaller subset? If yes, try a non-linear predictor with that subset.

**9** Do you have new ideas, time, computational resources, and enough examples? If yes,
compare several feature selection methods, including your new idea, correlation coefficients,
backward selection and embedded methods. Use linear and non-linear predictors.
Select the best approach with model selection.

**10** Do you want a stable solution (to improve performance and/or understanding)? If yes, subsample your data and redo your analysis for several subsets.

For a more comprehensive survey on feature selection methods and their recommended applications see: [[18]](#References) and [[this article](http://featureselection.asu.edu/algorithms.php)]

# References
<p style="font-size:0.6em;">
<a href="https://www.amazon.com/Pattern-Recognition-Learning-Information-Statistics/dp/0387310738">[1]</a>    : Bishop, C. (2006). Pattern recognition and machine learning<br>    
<a href="http://jmlr.csail.mit.edu/papers/volume3/guyon03a/guyon03a.pdf">[2]</a>    : Guyon, I. and Elissee, A. (2003). An Introduction to Variable and Feature Selection<br>
<a href="http://image-net.org/challenges/LSVRC/">[3]</a>: Russakovsky, O., Deng, J., Su, H., Krause, J., Satheesh, S., Ma, S., Huang, Z., Karpathy, A., Khosla, A., Bernstein, M., C. Berg, A. and Fei-Fei, L. (2015) ImageNet Large Scale Visual Recognition Challenge. IJCV, 2015<br>
<a href="https://snap.stanford.edu/data/wiki-meta.html">[4]</a>:Kossinets, G. (2010). Processed Wikipedia Edit History. Stanford large network dataset collection<br>
<a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3287491/">[5]</a>: Liu, Q., Sung, A. H., Chen, Z., Liu, J., Chen, L., Qiao, M., Deng, Y. (2011). Gene selection and classification for cancer microarray data based on machine learning and similarity measures.<br>
<a href="https://www.springer.com/gp/book/9780387848570">[6]</a>: Hastie, T., Tibshirani, R., and Friedman, J. H. (2009). The elements of Statistical learning: data mining, inference, and prediction.<br>
<a href="https://ieeexplore.ieee.org/document/4766926">[7]</a>: Trunk, G. V. (1979). A Problem of Dimensionality: A Simple Example<br>
    <a href="https://link.springer.com/chapter/10.1007/11494669_93">[8]</a>: Verleysen, M. and François, D. (2005). The Curse of Dimensionality in Data Mining and Time Series Prediction<br>
     <a href="   http://papers.nips.cc/paper/2020-on-discriminative-vs-generative-classifiers-a-comparison-of-logistic-regression-and-naive-bayes.pdf">[9]</a>: Ng, A. Y. and Jordan, M. I. (2001). On Discriminative vs. Generative Classifiers: A comparison of logistic regression and naive Bayes<br>
     <a href="   https://arxiv.org/abs/1601.07996">[10]</a>: Li, J., Cheng, K., Wang, S., Morstatter, F., Trevino, R. P., Tang, J., and Liu, H. (2017). Feature Selection: A Data Perspective<br>
    <a href="   https://www.sciencedirect.com/science/article/pii/S1088467X97000085">[11]</a>: Dash, M. and Liu, H. (1997). Feature selection for classification<br>
    <a href="https://www.researchgate.net/publication/2771488_Feature_Subset_Selection_as_Search_with_Probabilistic_Estimates">[12]</a>: Kohavi, R. (1994). Feature Subset Selection as Search with Probabilistic Estimates<br>
    <a href="https://www.researchgate.net/publication/288257551_Feature_selection_for_classification_A_review">[13]</a>: Tang, J., Alelyani, S., Liu, H. (2014). Feature selection for classification: A review<br>
    <a href="https://www.sciencedirect.com/science/article/pii/S0304397597001151">[14]</a>: Amaldi,E., Kann, V. (1998). On the approximability of minimizing nonzero variables or unsatisfied relations in linear systems<br>
    <a href="https://dl.acm.org/citation.cfm?id=2577699">[15]</a>:Chandrashekar, G., Sahin, F. (2014). A survey on feature selection methods<br>
      <a href="https://ieeexplore.ieee.org/document/1674939/">[16]</a>: Narendra, P. M., Fukunaga, K. (1977). A Branch and Bound Algorithm for Feature Subset Selection<br>
          <a href="https://www.amazon.com/Modern-Heuristic-Search-Methods-Rayward-Smith/dp/0471962805">[17]</a>: Rayward-Smith, V., Osman, I., Reeves, C. D., Smith, G. (1996). Modern Heuristic Search Methods<br>
      <a href="https://bib.irb.hr/datoteka/763354.MIPRO_2015_JovicBrkicBogunovic.pdf">[18]</a>: Jović, A., Brkić, K., and Bogunović, N. (2015). A review of feature selection methods with applications<br>

</p>
